In [192]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [193]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import theano.tensor as T
from theano import function
import seaborn as sns
from theano.tensor.shared_randomstreams import RandomStreams

In [194]:
# Import My own library for factor testing
import factorFilterFunctions as ff
#from config import *

In [195]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [196]:
path = ff.data_path # path

In [197]:
# --------------------------------------- Calculate Forward Adjusted Price ---------------------------------- #

In [198]:
# the necessary files
filenameAdjustFactor='LZ_GPA_CMFTR_CUM_FACTOR.csv'
filenamePirce='LZ_GPA_QUOTE_TCLOSE.csv'

In [199]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)
    AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=pd.read_csv(path+filenamePirce,infer_datetime_format=True,parse_dates=[0],index_col=0)
    PriceToAdj=PriceToAdj[:]

    #Calculate
    AdjFacforward = AdjFacBackward/AdjFacBackward.max()
    adjustedPrice = (AdjFacforward*PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [200]:
calAdjustedPrice()

Own_Factor_AdjustedPriceForward-1d


In [201]:
# --------------------------------------- Calculate ILLQ Factor(5-days average) ---------------------------------- #

In [202]:
# 
filenameClose = 'LZ_GPA_QUOTE_TCLOSE.csv'
filenameOpen = 'LZ_GPA_QUOTE_TOPEN.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'

In [203]:
def calcILLQ(): # rolling_window is set as 5 days
    openPrice = pd.read_csv(path+filenameOpen,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)
    volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if openPrice.shape != closePrice.shape:
        print openPrice.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = np.abs((closePrice - openPrice)/openPrice)/volume
        newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_ILLQ-1d'
        newdf.to_csv(path+'Own_Factor_ILLQ-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [204]:
#filenameILLQ = 'Own_Factor_ILLQ-1d.csv'
#openPrice = pd.read_csv(path+filenameILLQ,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [205]:
calcILLQ().tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_ILLQ-1d,,,,,,,,,,,,,,,,,,,,,
2017-04-21,1.133800e-07,7.193334e-08,NaN,1.351715e-07,1.588295e-07,1.635389e-07,NaN,NaN,NaN,2.212990e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,1.702447e-07,9.400260e-08,NaN,9.324179e-08,2.002378e-07,2.145127e-07,NaN,NaN,NaN,3.081750e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,1.833934e-07,9.051241e-08,NaN,1.311377e-07,1.597117e-07,1.714612e-07,NaN,NaN,NaN,3.228419e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,1.679786e-07,7.582259e-08,NaN,1.510474e-07,1.833934e-07,1.665644e-07,NaN,NaN,NaN,3.511408e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
# --------------------------------------- Calculate FCAP Adjusted Turnover Volume---------------------------------- #

In [207]:
# REMINDER: Code in this section can be used to neutralize any new factor! Use this to check some size-affected factor!!!

In [208]:
filenameTURNOVER = 'LZ_GPA_QUOTE_TVOLUME.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [209]:
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [210]:
TURNOVER = pd.read_csv(path+filenameTURNOVER,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

TURNOVER= simpleNormalize(TURNOVER)

FCAP1 = simpleNormalize(FCAP1 )

datelist = FCAP1.index.tolist()

In [211]:
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [212]:
adjustedTurnOver = neutralizeFactor(TURNOVER , FCAP1, datelist)

In [213]:
adjustedTurnOver.index.name = 'Own_Factor_AdjustedTurnOver-1d'
adjustedTurnOver.to_csv(path+'Own_Factor_AdjustedTurnOver-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [214]:
# --------------------------------------- Calculate FCAP Adjusted PB ---------------------------------- #

In [215]:
filenamePB = 'LZ_GPA_VAL_PB.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
PB = pd.read_csv(path+filenamePB ,infer_datetime_format=True,parse_dates=[0],index_col=0)
PB = simpleNormalize(PB)

In [216]:
adjustedPB = neutralizeFactor(PB , FCAP1, datelist)
adjustedPB.index.name = 'Own_Factor_AdjustedPB-1d'
adjustedPB.to_csv(path+'Own_Factor_AdjustedPB-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [217]:
pb1 = pd.read_csv(path+'Own_Factor_AdjustedPB-1d.csv',infer_datetime_format=True,parse_dates=[0],index_col=0)

In [218]:
pb1.columns

Index([u'000005.SZ', u'600601.SH', u'600602.SH', u'600651.SH', u'600652.SH',
       u'600653.SH', u'600654.SH', u'600656.SH', u'000004.SZ', u'000002.SZ',
       ...
       u'603139.SH', u'603232.SH', u'002865.SZ', u'300648.SZ', u'603096.SH',
       u'002866.SZ', u'300514.SZ', u'603920.SH', u'002867.SZ', u'603787.SH'],
      dtype='object', length=3299)

In [219]:
# --------------------------------------- Calculate  x-days return volatility  ---------------------------------- #

In [220]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [221]:
def calReturnVol(period): # 90days maybe good
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Volatility_%dd' % period
    newdf.to_csv(path+'Own_Factor_Volatility_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [222]:
calReturnVol(90).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_Volatility_90d,,,,,,,,,,,,,,,,,,,,,
2017-04-21,0.013276,0.013660,0.011544,0.017712,0.018463,0.012447,0.010129,NaN,0.018976,0.016977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,0.013532,0.014112,0.011498,0.017707,0.018709,0.013011,0.009614,NaN,0.018689,0.016992,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,0.013514,0.014062,0.011449,0.017683,0.018702,0.012989,0.009547,NaN,0.018650,0.016949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,0.013506,0.014062,0.011444,0.017719,0.019945,0.013052,0.009548,NaN,0.018650,0.016959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,0.012568,0.012803,0.010632,0.016380,0.019535,0.011887,0.000243,NaN,0.017085,0.015777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
# --------------------------------------- Calculate  x-days return above 20 days MA  ---------------------------------- #

In [224]:
def calAbove20MA(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    newdf = AdjPrice - AdjPrice.rolling(min_periods=20,window=period,center=False).mean()
    newdf = newdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf.index.name = 'Own_Factor_Above20MA_%dd' % period
    newdf.to_csv(path+'Own_Factor_Above20MA_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf 

In [225]:
calAbove20MA(20).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_Above20MA_20d,,,,,,,,,,,,,,,,,,,,,
2017-04-21,-0.112125,-0.147625,7.727152e-15,-0.166675,-1.028300,-0.101025,1.953993e-15,NaN,-1.488700,-0.124375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,-0.125525,-0.153725,7.727152e-15,-0.192225,-1.039725,-0.112875,1.953993e-15,NaN,-1.503575,-0.173175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,-0.141725,-0.160750,7.727152e-15,-0.226050,-1.039100,-0.124400,1.953993e-15,NaN,-1.487500,-0.213700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,-0.154350,-0.168000,7.727152e-15,-0.264750,-1.006600,-0.136450,1.953993e-15,NaN,-1.441425,-0.245425,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [226]:
# --------------------------------------- Calculate  ARoon  ---------------------------------- #

In [227]:
AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).iloc[-1000:]
adcopy = AdjPrice.copy()

In [228]:
adcopy.iloc[:,2].head()

Own_Factor_AdjustedPriceForward-1d
2013-03-19    3.91469
2013-03-20    4.02426
2013-03-21    4.08403
2013-03-22    4.10395
2013-03-25    4.01430
Name: 600602.SH, dtype: float64

In [229]:
def calAroon(data, l=20):
    datacopy = data.copy()
    for i in range(l,len(datacopy)):
        s = datacopy.iloc[i-l:i]
        #print s
        try:
            data.iloc[i] = pd.Timedelta(s.idxmax().date() - s.idxmin().date()).days/l
        except:
            data.iloc[i] = np.NaN
    return data

In [230]:
#aroonData = adcopy.iloc[:,:10].apply(calAroon,l=20)

In [231]:
# --------------------------------------- Calculate  daily deal Amount(yuan)   ---------------------------------- #

In [232]:
filenameTVolume = 'LZ_GPA_QUOTE_TVOLUME.csv' # 成交量
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [233]:
def calcDDA():
    turnOver = pd.read_csv(path+filenameTVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if turnOver.shape != closePrice.shape:
        print turnOver.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = turnOver * closePrice
        #newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_DDA-1d'
        newdf.to_csv(path+'Own_Factor_DDA-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [234]:
calcDDA()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_DDA-1d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,1.120256e+04,9.511504e+04,36486.408181,4.231575e+04,4.286417e+04,46478.301754,34848.379104,7.744337e+04,13739.751792,1.176296e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,1.104777e+04,1.539230e+05,77055.737351,1.189617e+05,3.607527e+04,36450.030077,66542.020171,2.682239e+04,24501.571164,2.070431e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,1.028649e+04,7.519487e+04,93782.209114,1.406231e+05,5.684882e+04,30255.485960,33524.838585,8.755773e+03,23666.695054,2.156412e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,4.150195e+04,1.162695e+05,81109.801898,5.668555e+04,5.576362e+04,34962.611051,62727.421697,3.152954e+04,17692.454404,2.093654e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,1.024396e+05,1.451684e+05,81562.183851,6.196563e+04,2.280886e+04,35335.227232,113085.368636,2.329403e+05,15792.835194,8.330268e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,2.546677e+04,7.329183e+04,66538.857640,7.007041e+04,2.777977e+04,30690.642067,53326.139704,4.126954e+05,51265.468366,7.055445e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,1.819887e+04,9.166338e+04,55987.950656,3.804263e+04,2.549850e+04,29026.524715,54666.832954,2.388097e+05,22423.051945,8.959669e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,2.032021e+04,1.246899e+05,40482.267958,1.689614e+05,2.643201e+04,36763.938919,77921.212387,3.383591e+05,49882.869110,1.099860e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,1.890111e+04,1.520970e+05,52393.353286,1.093032e+05,6.019900e+04,39863.661422,56831.726570,3.200100e+05,42039.785261,9.886799e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
filenameDDA = 'Own_Factor_DDA-1d.csv'
DDAdf = pd.read_csv(path+filenameDDA,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [236]:
def DDAaverage(period):
    DDAmean = DDAdf.rolling(min_periods=20,window=period,center=False).mean()
    DDAmean.index.name = 'Own_Factor_DDA-%dd' % period
    DDAmean.to_csv(path+'Own_Factor_DDA-%dd.csv' % period, na_rep='NaN',date_format='%Y%m%d')
    return DDAmean

In [237]:
DDAaverage(20)

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_DDA-20d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [239]:
def cal_factor_return(factor, stkreturn, factorname):
    '''
    To cal the return of the factor by group method( best(1/3) - worst(1/3))
    Output: Dataframe, the factor Return with only one value column
    Input:
    factor: Dataframe, the factor Value df, Note that shift(1) has been used before this function is used
    stkreturn:  Dataframe, the return of all stock in the market. No shift needed.
    factorname:  String, the name of the factor!
    '''
    factorReturn = pd.DataFrame(index=factor.index[1:-1], columns=[factorname], data =None , dtype =float)
    for date in factorReturn.index:
        factorSlice = factor.loc[date].dropna()
        stkreturnSlice = stkreturn.loc[date].dropna()
        intersection = list(set(factorSlice.index) & set(stkreturnSlice.index))
        factorSlice = factorSlice.loc[intersection]
        stkreturnSlice = stkreturnSlice.loc[intersection]
        
        q_min = factorSlice.quantile(0.33)
        q_max = factorSlice.quantile(0.66)
        q_min_univ = factorSlice[factorSlice<=q_min]
        q_max_univ = factorSlice[factorSlice>=q_max]
        q_min_return = (q_min_univ * stkreturnSlice.loc[q_min_univ.index]).sum() / q_min_univ.sum()
        q_max_return = (q_max_univ * stkreturnSlice.loc[q_max_univ.index]).sum() / q_max_univ.sum()
        #print q_min_return
        factorReturn.loc[date] = q_min_return - q_max_return
    return factorReturn

In [240]:
startTime =  datetime.strptime('20100101', '%Y%m%d')
endTime = datetime.strptime('20170328', '%Y%m%d')
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
filenamePB='LZ_GPA_VAL_PB.csv'  # 市净率
filenameBENCH = 'LZ_GPA_INDXQUOTE_CLOSE.csv'
ZZ500Index = '000905.SH' #   ZZ500 index code

In [241]:
pricedf = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
FCAPdf  = pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
PBdf = pd.read_csv(path+filenamePB,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
benchmarkdf = pd.read_csv(path+filenameBENCH,infer_datetime_format=True,parse_dates=[0],index_col=0)[ZZ500Index].loc[startTime:endTime].pct_change()
returndf = pricedf.pct_change()
FCAPdf = FCAPdf .shift(1)
PBdf = PBdf.shift(1)
FCAPdf.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
LZ_GPA_VAL_A_FCAP-d,,,,,,,,,,,,,,,,,,,,,
2017-03-22,631611.7134,985506.1506,825918.6874,815507.8476,756457.5094,670610.0417,1.316040e+06,NaN,329771.5014,2.056177e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-23,623050.7113,972336.8034,825918.6874,797829.5436,732637.1453,668863.6614,1.316040e+06,NaN,330518.1539,2.045498e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-24,619245.8215,965752.1298,825918.6874,793986.4341,721048.8600,661878.1401,1.316040e+06,NaN,324296.0501,2.082389e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-27,621148.2664,974531.6946,825918.6874,800135.4094,708816.7811,665370.9008,1.316040e+06,NaN,319982.0581,2.086272e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-28,616392.1541,976726.5858,825918.6874,803209.8970,703666.4321,661878.1401,1.316040e+06,NaN,315502.1434,2.058322e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
SizeReturn = cal_factor_return(FCAPdf, returndf , 'Size_Return')

In [243]:
SizeReturn.isnull().sum()

Size_Return    0
dtype: int64

In [244]:
PBReturn = cal_factor_return(PBdf, returndf , 'PB_Return')

In [245]:
PBReturn.isnull().sum()

PB_Return    0
dtype: int64

In [246]:
benchmarkdf = benchmarkdf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [247]:
benchmarkdf .isnull().sum()

0

In [248]:
returndf = returndf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [249]:
startOfMonthList, endOfMonthList = ff.getLastDayOfMonth(PBReturn.index)

In [250]:
len(startOfMonthList)

87

In [251]:
len(endOfMonthList)

87

In [252]:
datetuple = list(zip(startOfMonthList,endOfMonthList))

In [253]:
def calIdoVol(returnDF , factorReturnList , datetuple ,torelance = 0.05):
    '''
    function to calculate to idiosyncratic volatility.
    Output: Dataframe, The idiosyncratic volatility factor (same shape as ohter daily factor).
    Input : 
    returnDF: Dataframe, the data of the return of all stocks.
    factorReturn: List, the element is the factor-return dataframe. (Obtained by group-method so there is only one value \
    column(and one index column)
    for each dataframe).Note that this df has been shift(1) to ensure that future data is not used!
    datatuple: List, element is the tuple which is consisted of startTime and endTime. Usually zip by the startOfMonthList and endOfMonthList.
    torelance: float, to filter the Nan Value.
    '''
    startdf = pd.DataFrame()
    for i in datetuple:        
        returnDFSlice = returnDF.loc[i[0]:i[1]]
        tempdf= pd.DataFrame(index = returnDFSlice.index, columns = returnDFSlice.columns ,data = None ,dtype =float)
        newReturnSlice = returnDFSlice.loc[:,returnDFSlice.isnull().sum() < returnDFSlice.shape[1] * torelance]
        newReturnSlice = newReturnSlice.fillna(method = 'ffill').fillna(method = 'bfill')
        filterList = newReturnSlice.columns.tolist()
        mapfunction = map(lambda x: x.loc[i[0]:i[1]], factorReturnList)
        totaldf = pd.concat(mapfunction, axis=1) 
        for stk in filterList:
            result = sm.OLS(newReturnSlice[stk],totaldf).fit()
            tempdf[stk].loc[i[0]:i[1]] = np.std(result.resid) * np.sqrt(242)
        startdf = startdf.append(tempdf)
    return startdf        

In [254]:
dfList = [PBReturn,SizeReturn,benchmarkdf]

In [255]:
dd = calIdoVol(returndf , dfList,datetuple)

In [256]:
dd.index.name = 'idiosyncratic volatility'
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
dd.to_csv(path+'Own_Factor_Idiosyncratic_Volatility.csv', na_rep='NaN',date_format='%Y%m%d')

In [258]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [259]:
ddc =dd.tail(100)

In [260]:
filenameIDIVOL = 'Own_Factor_Idiosyncratic_Volatility.csv'
idio = pd.read_csv(path+filenameIDIVOL,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [261]:
idio.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
def adj_boxplot(factorData):
    '''To calculate  adjusted -boxplot winsorized data and then Normalize the outcome
    Output: Dataframe, the winsorized and normalized data
    Input: 
    factorData:Dataframe, raw data, can contain nan value
    '''
    copyData = factorData.copy()
    for i in copyData.index:
        temp = copyData.loc[i]
        x = temp.dropna().values
        if len(x) > 0:
            mc = sm.stats.stattools.medcouple(x)
            x.sort()
            q1 = x[int(0.25*len(x))]
            q3 = x[int(0.75*len(x))]
            iqr = q3-q1
            if mc >= 0:
                l = q1-1.5*np.exp(-3.5*mc)*iqr
                u = q3+1.5*np.exp(4*mc)*iqr
            else:
                l = q1-1.5*np.exp(-4*mc)*iqr
                u = q3+1.5*np.exp(3.5*mc)*iqr
            temp.loc[temp < l] = l
            temp.loc[temp > u] = u
            #factor_data.loc[i] = (temp-temp.mean())/temp.std()
    Trans = copyData.T
    return ((Trans  - Trans .mean(axis=0, skipna=True))/Trans .std(axis=0, skipna=True)).T

In [264]:
%mprun  addd = adj_boxplot(dd)

ERROR:root:Line magic function `%mprun` not found.


In [265]:
%lprun -f adj_boxplot adj_boxplot(idio)

In [266]:
# --------------------------------------- Calculate  some random factor  ---------------------------------- #

In [267]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
tradVol = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [268]:
def calrandfac(pricedf, factordf):
    sgndf= np.sign(pricedf.pct_change())
    tempo = factordf * sgndf
    tempo = tempo.ewm(ignore_na=True, min_periods=5, halflife = 5).mean()
    return tempo

In [269]:
overheatVol = calrandfac(closePrice, tradVol)

In [270]:
overheatVol

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
LZ_GPA_QUOTE_TVOLUME-d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,17362.211023,15599.700509,8196.017254,7338.990601,1224.180126,6698.553599,10590.838322,-14326.736494,3880.029943,28860.425910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,10402.596662,4943.392692,3961.613465,3901.998182,-714.132059,2198.381750,3396.387402,1668.029096,2044.179410,39472.459598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,11233.659633,12637.857932,1585.294374,9984.066901,970.715897,1740.174325,8768.956296,-15194.558737,3484.987477,50336.580403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,6374.148763,201.505827,-598.890251,3853.326015,-2465.104131,-2078.950153,2830.837631,-27492.719639,1318.541297,24416.840946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [271]:
overheatVol.index.name = 'OVER_HEAT_VOL'
overheatVol.to_csv(path+'Over_Heat_Volume.csv', na_rep='NaN',date_format='%Y%m%d')

In [272]:
# --------------------------------------- Calculate  skewness  ---------------------------------- #
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [273]:
# calculate N-days skewness of the price
def calSkewness(pricedf,period):
    df = pricedf.rolling(min_periods=250,window=period,center=False).skew()
    df.index.name =  'Own_Factor_Skewness_%dd' % period
    df.to_csv(path+'Own_Factor_Skewness_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return df

In [274]:
priceSkewness = calSkewness(closePrice,250)

In [275]:
# --------------------------------------- Calculate  TurnOver Rate Volatility ---------------------------------- #
filenameTOR= 'LZ_GPA_VAL_TURN.csv'
turnoverdf =  pd.read_csv(path+filenameTOR,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

In [276]:
def calToRvol(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Turnover_Volatility_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [277]:
newsdf = calToRvol(20)

In [278]:
newsdf.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_Turnover_Volatility_20D,,,,,,,,,,,,,,,,,,,,,
2017-04-21,1.209490,0.302999,NaN,NaN,0.619772,0.101383,NaN,NaN,NaN,0.190204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,1.207980,0.304779,NaN,NaN,0.622548,0.101890,NaN,NaN,NaN,0.187400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,1.209941,0.315521,NaN,NaN,0.644566,0.105002,NaN,NaN,NaN,0.191459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,1.216426,0.319385,NaN,NaN,0.746002,0.099492,NaN,NaN,NaN,0.192301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
NormalizedTOV = simpleNormalize(newsdf )
NormalizedFCAP = simpleNormalize(FCAP1 )

In [280]:
neutralizedTOV = neutralizeFactor(NormalizedTOV, NormalizedFCAP, NormalizedTOV.index)

In [281]:
neutralizedTOV.index.name = 'Own_Factor_ADJ_Turnover_Volatility_20D'
neutralizedTOV.to_csv(path+neutralizedTOV.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [283]:
def calToRVolD(period):
    newdf = turnoverdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf1 = turnoverdf.rolling(min_periods=500,window=500,center=False).mean()
    newdf = newdf / newdf1 -1
    newdf.index.name = 'Own_Factor_Turnover_Volatility_deviation_%dD' % period
    newdf.to_csv(path+'Own_Factor_Turnover_Volatility_deviation_%dD.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [284]:
df2 = calToRVolD(20)

In [287]:
df2 .tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_Turnover_Volatility_deviation_20D,,,,,,,,,,,,,,,,,,,,,
2017-04-21,-0.603979,-0.775334,NaN,NaN,-0.402041,-0.814174,NaN,NaN,NaN,-0.722289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,-0.603071,-0.769377,NaN,NaN,-0.417259,-0.811492,NaN,NaN,NaN,-0.735248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,-0.604074,-0.771577,NaN,NaN,-0.440183,-0.811307,NaN,NaN,NaN,-0.743625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,-0.607287,-0.771255,NaN,NaN,-0.414650,-0.808187,NaN,NaN,NaN,-0.749358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [288]:
NormalizedTOVD = simpleNormalize(df2)
neutralizedTOVD = neutralizeFactor(NormalizedTOVD, NormalizedFCAP, NormalizedTOVD.index)
neutralizedTOVD.index.name = 'Own_Factor_ADJ_Turnover_Volatility_Deviation_20D'
neutralizedTOVD.to_csv(path+neutralizedTOVD.index.name+'.csv',na_rep='NaN',date_format='%Y%m%d')

In [286]:
neutralizedTOVD .tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603139.SH,603232.SH,002865.SZ,300648.SZ,603096.SH,002866.SZ,300514.SZ,603920.SH,002867.SZ,603787.SH
Own_Factor_ADJ_Turnover_Volatility_Deviation_20D,,,,,,,,,,,,,,,,,,,,,
2017-04-21,-0.536355,-0.835447,NaN,NaN,-0.191273,-0.896362,NaN,NaN,NaN,-0.788605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-24,-0.523897,-0.814566,NaN,NaN,-0.208183,-0.878727,NaN,NaN,NaN,-0.815343,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-25,-0.510190,-0.803899,NaN,NaN,-0.235194,-0.859810,NaN,NaN,NaN,-0.843314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-26,-0.497961,-0.787793,NaN,NaN,-0.183573,-0.830767,NaN,NaN,NaN,-0.889647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
